## Gathering Data

### Source: Files on Hand
Udacity compiled, based on [this](https://www.rottentomatoes.com/top/bestofrt/) list from RottenTomatoes, a file with the Top 100 movies. 

#### Flat File Structure
Flat files contain tabular data in plain text format with one data record per line and each record or line having one or more fields. These fields are separated by delimiters, like commas, tabs, or colons.

**Advantages of flat files** include:

- They're text files and therefore human readable
- Lightweight
- Simple to understand
- Software that can read/write text files is ubiquitous, like text editors
- Great for small dataset

**Disadvantages of flat files**, in comparison to relational databases, for example, include:

- Lack of standards
- Data redundancy
- Sharing data can be cumbersome
- Not great for large datasets

#### Flat Files in Python
Pandas is especially suited to read tabular data.

[`pandas.read_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) can handle all kinds of flat files, including TSV files by changing the parameters. Try importing the .tsv file based on the Top 100 Movies from RottenTomatoes. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
df = pd.read_csv('support-files/02_Gathering-Data/bestofrt.tsv', sep='\t')
df.head()

,ranking,critic_score,title,number_of_critic_ratings
0,1,99,The Wizard of Oz (1939),110
1,2,100,Citizen Kane (1941),75
2,3,100,The Third Man (1949),77
3,4,99,Get Out (2017),282
4,5,97,Mad Max: Fury Road (2015),370


### Source: Web Scraping
Next, we'd like to get Rotten Tomatoes' audience scores and the number of audience reviews to add to our dataset. However, this is not easily accessible from the website and to get this data we will need to do web scraping, which allows us to extract data from websites using code.

In order to do that, we'll use [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/).

Read [this](https://medium.com/towards-data-science/ethics-in-web-scraping-b96b18136f01) article on the ethical issues involved in web scraping.

In [1]:
# continue here!